In [1]:
import random
random.seed(123)
import numpy as np
import pandas as pd
np.random.seed(123)

In [2]:
# read txt file and convert it into pandas dataframe.
txt = ''
with open('seeds_dataset.txt') as f :
    txt = f.read()
    

In [3]:
data = []
for line in txt.split('\n') :
    data.append([word for word in line.split('\t') if word != ''])

In [4]:
df = pd.DataFrame(data,columns=['area','perimeter','compactness','kernel_len','kernel_wid','coef_asymmetry','kernel_groove_len'
                               ,'wheat_type'],copy=True)

In [5]:
df.wheat_type.unique()

array(['1', '2', '3'], dtype=object)

In [6]:
df.shape

(210, 8)

In [7]:
def get_normal_random_weight() :
    return random.uniform(0, 1)
get_normal_random_weight()

0.052363598850944326

In [8]:
def build_single_neuron(inp) :
    neuron = {}
    neuron['weights'] = [ random.uniform(0, 1) for i in range(inp+1) ]
    return neuron

In [42]:
def build_single_hidden_layer(no_of_neurons,no_of_inputs) :
    layer = []
    for i in range(no_of_neurons) :
        layer.append(build_single_neuron(no_of_inputs))
    return layer

[{'weights': [0.06263564431359725,
   0.25336194995859596,
   0.168020375038908,
   0.4160567101659304,
   0.4249764758602258]},
 {'weights': [0.6070585475338274,
   0.49680968876011555,
   0.16476953922391535,
   0.046621071319652985,
   0.5247721982443964]},
 {'weights': [0.7638566846893943,
   0.9268939013125862,
   0.8396383240376201,
   0.3027528040099047,
   0.8656732430775638]}]

In [44]:
def build_hidden_layers(layers,no_of_neurons,no_of_inputs) :
    network = []
    for i in range(layers) :
        network.append(build_single_hidden_layer(no_of_neurons,no_of_inputs))
    return network

[[{'weights': [0.4673915447602902,
    0.2824470291123572,
    0.09215872811695913,
    0.4427124158506194,
    0.5449613597795614]},
  {'weights': [0.27929936950932854,
    0.19529882461359027,
    0.9408904680768644,
    0.06777491893982879,
    0.939392256680005]},
  {'weights': [0.40641295759207385,
    0.7915177900236534,
    0.11094804898017563,
    0.5700076403666973,
    0.761696224317614]}],
 [{'weights': [0.4360117280281445,
    0.35646624232715174,
    0.4932188103687801,
    0.1944599146366902,
    0.2116300536464455]},
  {'weights': [0.24131152758298502,
    0.03755665752648618,
    0.9100241304052978,
    0.07343992146589373,
    0.9271381201286817]},
  {'weights': [0.010956133993142214,
    0.6562134498500771,
    0.5469276311244807,
    0.04184170011186694,
    0.9414667729029921]}]]

In [48]:
def build_output_layer(out_neuron,no_of_inputs) :
    return build_hidden_layers(1,out_neuron,no_of_inputs)

[[{'weights': [0.808001576013212,
    0.465399007301589,
    0.6478904242823934,
    0.2842166187874783,
    0.5385051352053422]},
  {'weights': [0.5626741945572333,
    0.5865009997208089,
    0.7860964907016605,
    0.3386246278783821,
    0.16918262868879097]},
  {'weights': [0.3748940158813264,
    0.5859276212515593,
    0.4908297196196507,
    0.37245172824233197,
    0.42137498902751547]}]]

In [11]:
def calculate_activation_value(weights,inputs) :
    activation = weights[-1]
    for i in range(len(weights)-1) :
        activation += weights[i] * inputs[i]
    return activation

In [12]:
def transfer(act) :
    return 1/(1+e^(-act))

In [13]:
def forward_propagate(network,row) :
    inputs = row
    for layer in network :
        new_inputs = []
        for neuron in layer :
            activation = calculate_activation_value(neuron['weights'],inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [47]:
inp = df.shape[1] -1 
hidden_layers = 2    # two hidden layers.
hidden_neurons = 7 

out = len(df.wheat_type.unique())
print(inp,hidden_layers,hidden_neurons,out)

def build_neural_network(inp,hidden_layers,hidden_neurons,out) :
    network = build_hidden_layers(hidden_layers,hidden_neurons,inp)
    network.append(build_output_layer(out,hidden_neurons))
    return network
build_neural_network(inp,hidden_layers,hidden_neurons,out)

7 2 7 3


[[{'weights': [0.16335569610662282,
    0.02671482607993625,
    0.7287206950121947,
    0.764372512055565,
    0.9009222434862069,
    0.6724659589033983,
    0.78009977335241,
    0.365520098686237]},
  {'weights': [0.10081332278536825,
    0.9999133401695608,
    0.6046932794257612,
    0.2704503396247001,
    0.8927630924752599,
    0.07359212905995738,
    0.05678103055556505,
    0.2548510682668663]},
  {'weights': [0.1300911773150547,
    0.4107476777789437,
    0.6210782696305431,
    0.5194195120962015,
    0.5537166138297663,
    0.1193780129122054,
    0.5349347004365425,
    0.9751611718264387]},
  {'weights': [0.32978934263362525,
    0.799247419841678,
    0.6379859959069121,
    0.7620594872751336,
    0.6579474755258273,
    0.43950779703815535,
    0.23497705910616173,
    0.09356901674573537]},
  {'weights': [0.11324530556289636,
    0.10092621246200106,
    0.7673499393533436,
    0.8373133122046096,
    0.8176168570544177,
    0.8847971625246096,
    0.3587685148639

In [36]:
df.shape

(210, 8)

In [37]:
df.wheat_type.unique()

array(['1', '2', '3'], dtype=object)